This notebook is intended to demonstrate how to use github events data to generate new features. It will compare timestamps of code commits between pairs of projects to extract information relating to whether one project's code updates tend to trigger updates of another. This will typically indicate some dependency, although with a great deal of resulting noise.

The resulting features will have two primary values: a simple pre/post ratio, and percentage of A's commits that occur in some interval after B has made a commit. There are surely other combinations and time values that could be explored.

First, we will join all of A's commits with all of B's commits, and keep the ones that occur within T days (we can take 30 days as one option). The pre/post ratio will simply count the number of times A's commit "i" occurs after B's commit "j", and likewise count how often j occurs after i. This will give a ratio indicating which event is more likely.

The percentage of A's commits within a window of B's commits is similar, but dependent on the total count of A's commits. For each pair of projects, we will count how often each commit from A occurs within some commit of B. Then we take the fraction of those over the total count of A's commits. We will do this for all project pairs we are looking to score.

In [150]:
import pandas as pd
import numpy as np
import polars as pl
from google.colab import auth
from google.cloud import bigquery

auth.authenticate_user()


# Set up dataset and preprocessing

In [168]:
# All we need for this exercise is the graph itself.

df = pl.read_parquet('hf://datasets/evalscience/deepfunding-mini/train.parquet')
df = df.to_pandas()
df = df[['project_a','project_b', 'weight_a', 'weight_b']]


In [169]:
# First we need the artifact ids for each project so we can join them to the
# events data.
# NOTE: This table may be out of date.


%%bigquery project_artifacts --project opensource-observer

select *
from `oso_production.artifacts_v1` e
where artifact_source = "GITHUB"




Query is running:   0%|          |

Downloading:   0%|          |

In [170]:
all_project_urls = list(set(df.project_a.values) & set(df.project_b.values))
project_artifacts = project_artifacts[project_artifacts.artifact_url.isin(all_project_urls)]

artifact_names = dict(list(zip(project_artifacts.artifact_id.values, project_artifacts.artifact_name.values)))
project_artifacts.head()

,artifact_id,artifact_source_id,artifact_source,artifact_namespace,artifact_name,artifact_url
28119,6-dQlK0cD_GCJcmMSlIf19E1aKPfgrXZ9TD_PmCOMKY=,434708679,GITHUB,vitest-dev,vitest,https://github.com/vitest-dev/vitest
55604,C_p1SL6CkELtDbMr85mtQHWzXcmJneIlj7UFFnzhFBg=,27038309,GITHUB,colorjs,color-name,https://github.com/colorjs/color-name
61686,z-hapr-nDJE_hZl_lgYnnOc8hZW5dZL_tJ1XqD6y9B0=,12519245,GITHUB,kaelzhang,node-ignore,https://github.com/kaelzhang/node-ignore
113150,DPwCWCf0GjbwbI3Nu7XbJT-uan6LRQfEWy3O3F6uve0=,64622042,GITHUB,pnpm,cmd-shim,https://github.com/pnpm/cmd-shim
126568,2DbwLXDOofsGEJe74sBwCFMZdpzfoH-aeuCZ1caHLuM=,129558668,GITHUB,nomicfoundation,hardhat,https://github.com/nomicfoundation/hardhat


In [132]:
# This will give us all the events data we have to work with.
# Note: It's recommended to download these results to avoid multiple scans
# over a large table.

%%bigquery code_commits --project opensource-observer

select *
from `oso_production.timeseries_events_by_artifact_v0` e
where
  event_type = 'COMMIT_CODE'


# code_commits = pd.read_csv('code_commit.csv')
# code_commits.to_csv('code_commit.csv', index=False)


Query is running:   0%|          |

Downloading:   0%|          |

In [164]:

code_commits['project_name'] = code_commits.to_artifact_id.map(artifact_names)
print(code_commits.shape)
code_commits.head()

(6974090, 8)


,time,to_artifact_id,from_artifact_id,event_type,event_source_id,event_source,amount,project_name
0,2024-11-25 09:11:43+00:00,1CzwtiABQnHiZ7zor7peY3bpYWKuok2xpDwjJhKqSHA=,TbO0II6XaF9T1JG4AVGamLHoYxz6jHO0nLifHgix9q4=,COMMIT_CODE,21383680730,GITHUB,1.0,NaN
1,2022-05-31 07:41:46+00:00,1TAsQo6AzN4aV3M8895qGCGXUj4n6E4QS5wuz8i1m0w=,PTddE3bBvwBAQD9yOqP-Psp8D93ObAyrzlroeZb7Pvk=,COMMIT_CODE,10026584702,GITHUB,1.0,NaN
2,2024-08-06 12:46:20+00:00,dgCmpNNSMNgI_DiE_5Ule1csx3ZrridU0QJCzxtpTLE=,2D2CmWPnRvmMHp-lW0wq8OwZZsvkaS0PF3GioNgtVaw=,COMMIT_CODE,19647626010,GITHUB,1.0,reth
3,2020-07-30 15:14:48+00:00,rY2OAklv06kR7SXDr4gi1va-8kAfKZLrkEbN_sL7J5w=,fwxHQ-bvzqHr-yXiy9tbz47C7dnysqFrOhZ-GlOP8XE=,COMMIT_CODE,5456330379,GITHUB,1.0,NaN
4,2019-01-06 17:47:35+00:00,xBS6uxaL8CXYuWZHnRA4B4u5ltlXrOM-cL5YdaGQiB0=,8eCP_figoj8A__4ZAfTuR3gM_RoBaU3rwSGVK00VG5c=,COMMIT_CODE,3184897293,GITHUB,1.0,NaN


In [165]:
commit_counts = code_commits.groupby('project_name', as_index=False)['amount'].count()


In [180]:
project_urls = list(set(df.project_a.values) & set(df.project_b.values))
project_names = dict(list(zip(project_urls, map(lambda s: s.split('/')[4], project_urls))))
project_ids = {pname: i for i, pname in enumerate(project_names.values())}


df['project_name_a'] = df.project_a.map(project_names)
df['project_name_b'] = df.project_b.map(project_names)

# This table for artifact names appears to be out of date...
df_original = df
df = df[~df.project_name_a.isna()]
df = df[~df.project_name_b.isna()]

project_full_names = dict(list(set(list(zip(df.project_name_a.values, df.project_a.values)) + list(zip(df.project_name_b.values, df.project_b.values)))))

df['a_id'] = df['project_name_a'].map(project_ids).astype(int)
df['b_id'] = df['project_name_b'].map(project_ids).astype(int)

proj_commits = code_commits[code_commits.project_name.isin(project_ids)]
proj_commits = proj_commits.merge(commit_counts, on='project_name')

proj_commits['project_id'] = proj_commits['project_name'].map(project_ids)
proj_commits['count'] = proj_commits['amount_y']
del proj_commits['amount_y']

proj_commits['timestamp'] = proj_commits.time.apply(lambda t: pd.Timestamp(t))
proj_commits['year'] = proj_commits.timestamp.apply(lambda t: t.year)
proj_commits['month'] = proj_commits.timestamp.apply(lambda t: t.month)

print(proj_commits.shape)
proj_commits.head()

(122877, 13)


,time,to_artifact_id,from_artifact_id,event_type,event_source_id,event_source,amount_x,project_name,project_id,count,timestamp,year,month
0,2024-08-06 12:46:20+00:00,dgCmpNNSMNgI_DiE_5Ule1csx3ZrridU0QJCzxtpTLE=,2D2CmWPnRvmMHp-lW0wq8OwZZsvkaS0PF3GioNgtVaw=,COMMIT_CODE,19647626010,GITHUB,1.0,reth,10,7283,2024-08-06 12:46:20+00:00,2024,8
1,2023-04-05 11:52:56+00:00,HTbb-R2mDywHM874Rlo5u-nN-VlvjpMk8q-tuPs_yFg=,QL6cREWzB7gAxKlLRqbNsky0R8ZMFid3q73I_8t9r4E=,COMMIT_CODE,13197695441,GITHUB,1.0,remix-project,99,10292,2023-04-05 11:52:56+00:00,2023,4
2,2023-01-05 20:41:55+00:00,nQ-L4-5oHa0R-wQedIdYB0OBzZcNz5FH3QgWEbn9dDI=,4YGs2bLo_5kECHFa3mCS6ortpVudNMMa_pSOgJhgKEM=,COMMIT_CODE,12193316867,GITHUB,1.0,react,8,3357,2023-01-05 20:41:55+00:00,2023,1
3,2016-02-29 14:05:40+00:00,0mjl8VhWsui_6TEZZnbQzyf8h1A9bOioIlK17p0D5hI=,1y7Pig-2ucYQY7_Z5BE6L5MiJ6fg_6MfW8X5vPaRNag=,COMMIT_CODE,998455128,GITHUB,1.0,go-ethereum,61,8059,2016-02-29 14:05:40+00:00,2016,2
4,2016-05-16 22:31:27+00:00,SJyWDLBT3mf86EJzaD9OfwGYfGxnA33_uUJlQfT5P4E=,MSUulyDEuz4toAG0NMJF3P4PRfg5PPutXTPp0nWI3SQ=,COMMIT_CODE,1114424679,GITHUB,1.0,quic-go,88,6454,2016-05-16 22:31:27+00:00,2016,5


In [181]:
ndays = 14
pdelta = pd.Timedelta(str(ndays) + ' days')

def filter_within_window(td):
    return not (td < -pdelta or td > pdelta)


# Merge and Compute Features

In [182]:
# Note: This merge takes some shortcuts, due to the size of the cross-commit
# table, such as only joining on the same month.
# We would want to repeat the process for each year, since it uses 2024 only.

import warnings
warnings.filterwarnings('ignore')

year_to_merge = 2024

proj_commits['project_month'] = proj_commits.project_id * 120 + (proj_commits.year-2015)*12 + (proj_commits.month-1)

df_commits = df[['a_id', 'b_id']].merge(proj_commits.query(f'year=={year_to_merge}')[['project_id', 'timestamp', 'count', 'project_month']], left_on='a_id', right_on='project_id', how='left')

df_commits = df_commits[~df_commits.a_id.isna()]
df_commits = df_commits[~df_commits.b_id.isna()]
df_commits = df_commits[~df_commits.project_id.isna()]

print(df_commits.shape,df_commits.columns)

proj_commits.set_index(['project_month'], inplace=True)
df_commits['project_month'] = df_commits.b_id * 120 + df_commits.project_month - df_commits.project_id * 120
df_commits['project_month'] = df_commits['project_month'].astype(int)
df_commits.set_index(['project_month'], inplace=True)

del df_commits['project_id']

df_cross_commits = df_commits.merge(proj_commits.query(f'year=={year_to_merge}')[['timestamp', 'count']], left_index=True, right_index=True) # left_on=['b_id', 'year', 'month'], right_on=['project_id', 'year', 'month'], [['project_id', 'timestamp', 'count', 'year', 'month']]
df_cross_commits = df_cross_commits.rename(columns={'timestamp_x': 'timestamp_a', 'timestamp_y': 'timestamp_b', 'count_x': 'count_a', 'count_y':'count_b'})
df_cross_commits[['count_a','count_b']] = df_cross_commits[['count_a','count_b']].astype(int)

print(df_cross_commits.shape)

df_cross_commits = df_cross_commits[~df_cross_commits.timestamp_a.isna()]
df_cross_commits = df_cross_commits[~df_cross_commits.timestamp_b.isna()]

df_cross_commits['timedelta'] = df_cross_commits['timestamp_a'] - df_cross_commits['timestamp_b']
df_commits_close = df_cross_commits[df_cross_commits.timedelta.apply(filter_within_window)]

df_commits_close['a_pre_b'] = df_commits_close.timedelta < pd.Timedelta('0 days')
df_commits_close['a_post_b'] = df_commits_close.timedelta >= pd.Timedelta('0 days')

df_commits_close['ts_a'] = df_commits_close.timestamp_a.astype(int)
df_commits_close['ts_b'] = df_commits_close.timestamp_b.astype(int)
df_commits_close.reset_index(drop=True, inplace=True)

df_commits_close.head()


(384726, 6) Index(['a_id', 'b_id', 'project_id', 'timestamp', 'count', 'project_month'], dtype='object')
(6516588, 6)


,a_id,b_id,timestamp_a,count_a,timestamp_b,count_b,timedelta,a_pre_b,a_post_b,ts_a,ts_b
0,29,16,2024-02-02 06:25:42+00:00,428,2024-02-08 15:31:58+00:00,2439,-7 days +14:53:44,True,False,1706855142000000000,1707406318000000000
1,29,16,2024-02-02 06:25:42+00:00,428,2024-02-14 12:11:57+00:00,2439,-13 days +18:13:45,True,False,1706855142000000000,1707912717000000000
2,29,16,2024-02-02 06:25:42+00:00,428,2024-02-15 10:45:13+00:00,2439,-14 days +19:40:29,True,False,1706855142000000000,1707993913000000000
3,29,16,2024-02-02 06:25:42+00:00,428,2024-02-12 10:42:20+00:00,2439,-11 days +19:43:22,True,False,1706855142000000000,1707734540000000000
4,29,16,2024-02-02 06:25:42+00:00,428,2024-02-15 10:45:13+00:00,2439,-14 days +19:40:29,True,False,1706855142000000000,1707993913000000000


In [183]:
# This code counts the percentage of commits project A makes within a week of any commit for project B, and vice versa.

horizon = 7
df_commits_close['a_within_t'] = (df_commits_close.timedelta > pd.Timedelta('0 days')) & (df_commits_close.timedelta <= pd.Timedelta(f'{horizon} days'))
df_commits_close['b_within_t'] = (df_commits_close.timedelta < pd.Timedelta('0 days')) & (df_commits_close.timedelta >= pd.Timedelta(f'-{horizon} days'))

a_commit_within_t = df_commits_close.groupby(['ts_a', 'a_id', 'b_id', 'count_a', 'count_b'], as_index=False)['a_within_t'].sum()
b_commit_within_t = df_commits_close.groupby(['ts_b', 'a_id', 'b_id', 'count_a', 'count_b'], as_index=False)['b_within_t'].sum()

a_commit_within_t['a_within_t'] = a_commit_within_t['a_within_t'].fillna(0) > 0
b_commit_within_t['b_within_t'] = b_commit_within_t['b_within_t'].fillna(0) > 0

a_commit_within_t = a_commit_within_t.groupby(['a_id', 'b_id', 'count_a'], as_index=False)['a_within_t'].sum()
b_commit_within_t = b_commit_within_t.groupby(['a_id', 'b_id', 'count_b'], as_index=False)['b_within_t'].sum()

commit_within_t = (a_commit_within_t.merge(b_commit_within_t, on=['a_id', 'b_id'], how='outer'))
commit_within_t[['a_within_t', 'b_within_t']] = commit_within_t[['a_within_t', 'b_within_t']].fillna(0)

commit_within_t['percent_a_within_t'] = (commit_within_t.a_within_t /commit_within_t.count_a)
commit_within_t['percent_b_within_t'] = (commit_within_t.b_within_t /commit_within_t.count_b)

commit_within_t.head()

,a_id,b_id,count_a,a_within_t,count_b,b_within_t,percent_a_within_t,percent_b_within_t
0,1,3,7,1,5,4,0.142857,0.800000
1,1,26,7,7,2760,10,1.000000,0.003623
2,1,33,7,5,5,3,0.714286,0.600000
3,1,37,7,7,25,16,1.000000,0.640000
4,2,1,25,5,7,6,0.200000,0.857143


In [184]:
# Compute the simple pre/post ratio.

prepost = df_commits_close.groupby(['a_id', 'b_id'], as_index=False)[['a_pre_b','a_post_b']].sum()
prepost['pre_post_ratio'] = (prepost.a_post_b + 1) / (prepost.a_pre_b+ prepost.a_post_b + 1)
prepost.head()


,a_id,b_id,a_pre_b,a_post_b,pre_post_ratio
0,1,3,23,2,0.115385
1,1,26,36,141,0.797753
2,1,33,6,12,0.684211
3,1,37,66,49,0.431034
4,2,1,30,68,0.696970


# Tidy up and save the results.

In [185]:

final_df = df.merge(commit_within_t, on=['a_id', 'b_id'], how='left').fillna(0)
final_df = final_df.merge(prepost, on=['a_id', 'b_id'], how='left')
final_df['pre_post_ratio'] = final_df['pre_post_ratio'].fillna(0.5)
final_df = final_df.fillna(0.0)
for k in ['a_id', 'b_id', 'project_name_a', 'project_name_b']: del final_df[k]

final_df = final_df.rename(columns={'count_a': 'commit_count_a', 'count_b': 'commit_count_b'})

final_df.to_csv('code_commit_cross_analysis.csv', index=False)

from google.colab import files
files.download('code_commit_cross_analysis.csv')
final_df.head()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,project_a,project_b,weight_a,weight_b,commit_count_a,a_within_t,commit_count_b,b_within_t,percent_a_within_t,percent_b_within_t,a_pre_b,a_post_b,pre_post_ratio
0,https://github.com/prettier-solidity/prettier-...,https://github.com/nomicfoundation/hardhat,0.101669,0.898331,428.0,50.0,2439.0,138.0,0.116822,0.056581,1700.0,1669.0,0.495549
1,https://github.com/prettier-solidity/prettier-...,https://github.com/consensys/teku,0.669446,0.330554,428.0,53.0,5324.0,266.0,0.123832,0.049962,1100.0,973.0,0.469624
2,https://github.com/prettier-solidity/prettier-...,https://github.com/ethereum/solidity,0.449022,0.550978,428.0,50.0,18867.0,160.0,0.116822,0.008480,1574.0,1707.0,0.520414
3,https://github.com/prettier-solidity/prettier-...,https://github.com/ethereum/remix-project,0.498396,0.501604,428.0,53.0,10292.0,235.0,0.123832,0.022833,4807.0,4557.0,0.486706
4,https://github.com/prettier-solidity/prettier-...,https://github.com/ethereum/go-ethereum,0.272503,0.727497,428.0,52.0,8059.0,326.0,0.121495,0.040452,1287.0,1452.0,0.530292


In [186]:
np.corrcoef(final_df.query('percent_b_within_t>0').percent_b_within_t.values, final_df.query('percent_b_within_t>0').weight_a.values)
np.corrcoef(final_df.query('percent_a_within_t>0').percent_a_within_t.values, final_df.query('percent_a_within_t>0').weight_b.values)


array([[1.        , 0.19126904],
       [0.19126904, 1.        ]])

In [187]:
np.corrcoef(final_df.pre_post_ratio.values, final_df.weight_b.values)


array([[1.        , 0.12927838],
       [0.12927838, 1.        ]])